# JAAD Preprocessing stages... 

In [1]:
import cv2
import numpy as np
import os 
from jaad_data import JAAD
import os
import pandas as pd
import pickle
import csv
import tqdm

In [2]:
print(os.getcwd())

/Users/rajarathnambalakrishnan/Documents/GitHub/Volvo-DataX/Berkeley/Data


In [3]:
jaad_path = os.getcwd()
imdb = JAAD(data_path=jaad_path)

In [4]:
#imdb.extract_and_save_images()

video_0001
[####################] 99.83% 

video_0002
[####################] 99.52% 

video_0003
[####################] 99.52% 

video_0004
[####################] 99.44% 

video_0005
[####################] 99.58% 

video_0006
[####################] 99.70% 

video_0007
[####################] 99.17% 

video_0008
[####################] 99.33% 

video_0009
[####################] 99.17% 

video_0010
[####################] 98.89% 

video_0011
[####################] 99.63% 

video_0012
[####################] 99.17% 

video_0013
[####################] 99.33% 

video_0014
[####################] 99.63% 

video_0015
[####################] 99.76% 

video_0016
[####################] 99.52% 

video_0017
[####################] 99.63% 

video_0018
[####################] 99.74% 

video_0019
[####################] 99.79% 

video_0020
[####################] 99.81% 

video_0021
[####################] 99.44% 

video_0022
[####################] 99.76% 

video_0023
[####################] 99.58% 

video_0024


In [5]:
all_vid_ids = imdb._get_video_ids_split(image_set='all', subset='default')
train_vid_ids = imdb._get_video_ids_split(image_set='train', subset='default')
val_vid_ids = imdb._get_video_ids_split(image_set='val', subset='default')
test_vid_ids =imdb._get_video_ids_split(image_set='test', subset='default')

In [6]:
print(len(all_vid_ids))
print(len(train_vid_ids))
print(len(val_vid_ids))
print(len(test_vid_ids))

323
177
29
117


In [14]:
#imdb.get_data_stats()

---------------------------------------------------------
Generating database for jaad
Getting annotations for video_0001
Getting annotations for video_0002
Getting annotations for video_0003
Getting annotations for video_0004
Getting annotations for video_0005
Getting annotations for video_0006
Getting annotations for video_0007
Getting annotations for video_0008
Getting annotations for video_0009
Getting annotations for video_0010
Getting annotations for video_0011
Getting annotations for video_0012
Getting annotations for video_0013
Getting annotations for video_0014
Getting annotations for video_0015
Getting annotations for video_0016
Getting annotations for video_0017
Getting annotations for video_0018
Getting annotations for video_0019
Getting annotations for video_0020
Getting annotations for video_0021
Getting annotations for video_0022
Getting annotations for video_0023
Getting annotations for video_0024
Getting annotations for video_0025
Getting annotations for video_0026
Get

Getting annotations for video_0236
Getting annotations for video_0237
Getting annotations for video_0238
Getting annotations for video_0239
Getting annotations for video_0240
Getting annotations for video_0241
Getting annotations for video_0242
Getting annotations for video_0243
Getting annotations for video_0244
Getting annotations for video_0245
Getting annotations for video_0246
Getting annotations for video_0247
Getting annotations for video_0248
Getting annotations for video_0249
Getting annotations for video_0250
Getting annotations for video_0251
Getting annotations for video_0252
Getting annotations for video_0253
Getting annotations for video_0254
Getting annotations for video_0255
Getting annotations for video_0256
Getting annotations for video_0257
Getting annotations for video_0258
Getting annotations for video_0259
Getting annotations for video_0260
Getting annotations for video_0261
Getting annotations for video_0262
Getting annotations for video_0263
Getting annotations 

In [4]:
train_opts = {'fstride': 1,
             'sample_type': 'all',
             'subset': 'high_visibility',
             'data_split_type': 'default',
             'seq_type': 'trajectory',
             'height_rng': [0, float('inf')],
             'squarify_ratio': 0,
             'min_track_size': 0,
             'random_params': {'ratios': None,
                               'val_data': True,
                               'regen_data': True},
             'kfold_params': {'num_folds': 5, 'fold': 1}}

In [5]:
# generate annotation data for training set...

imdb.get_detection_data(image_set = 'train', 
                                method = 'yolo3', 
                                occulusion_type = 'full', 
                                file_path = '/Users/rajarathnambalakrishnan/Documents/GitHub/Volvo-DataX/Berkeley/Data/',
                                **train_opts)

---------------------------------------------------------
Generating database for jaad
jaad database loaded from /Users/rajarathnambalakrishnan/Documents/GitHub/Volvo-DataX/Berkeley/Data/data_cache/jaad_database.pkl
Number of unique pedestrians 1314 
Number of samples 139065 
Data generated for YOLO3


('/Users/rajarathnambalakrishnan/Documents/GitHub/Volvo-DataX/Berkeley/Data/yolo3_train.txt',
 '/Users/rajarathnambalakrishnan/Documents/GitHub/Volvo-DataX/Berkeley/Data/mapping_yolo3.txt')

In [8]:
# process the annotation data into a list...
with open('/Users/rajarathnambalakrishnan/Documents/GitHub/Volvo-DataX/Berkeley/Data/yolo3_train.txt', 'r') as csvfile:
    list_arr = []
    reader = csv.reader(csvfile, delimiter=' ')

    for row in reader:
        list_arr.append(row)

img_list = []
for i in tqdm.tqdm(list_arr):
    img_list.append(i[0])

# saving image list for tfrecord processing 

with open("train_img_list.txt", "wb") as fp:   #Pickling
    pickle.dump(img_list, fp)
with open("train_annotations.txt", "wb") as fp1:   #Pickling
    pickle.dump(list_arr, fp1) 

100%|██████████| 37110/37110 [00:00<00:00, 2822979.51it/s]


In [9]:
# convert annotations into .tfrecord format...
!python to_tfrecord.py \
  --image_list 'train_img_list.txt' \
  --annotations 'train_annotations.txt' \
  --output_file train.tfrecord

I0302 21:22:58.889111 4833693120 to_tfrecord.py:101] Class mapping loaded
I0302 21:22:58.944694 4833693120 to_tfrecord.py:115] Image list loaded: 37110
100%|████████████████████████████████████| 37110/37110 [06:04<00:00, 101.87it/s]
I0302 21:29:03.249040 4833693120 to_tfrecord.py:126] Done


In [5]:
val = imdb.get_detection_data(image_set = 'val', 
                                method = 'yolo3', 
                                occulusion_type = 'full', 
                                file_path = '/Users/rajarathnambalakrishnan/Documents/GitHub/Volvo-DataX/Berkeley/Data/',
                                **train_opts)

---------------------------------------------------------
Generating database for jaad
jaad database loaded from /Users/rajarathnambalakrishnan/Documents/GitHub/Volvo-DataX/Berkeley/Data/data_cache/jaad_database.pkl
Number of unique pedestrians 196 
Number of samples 22498 
Data generated for YOLO3


In [6]:
# process the annotation data into a list...
with open('/Users/rajarathnambalakrishnan/Documents/GitHub/Volvo-DataX/Berkeley/Data/yolo3_val.txt', 'r') as csvfile:
    list_arr = []
    reader = csv.reader(csvfile, delimiter=' ')

    for row in reader:
        list_arr.append(row)

img_list = []
for i in tqdm.tqdm(list_arr):
    img_list.append(i[0])

# saving image list for tfrecord processing 

with open("val_img_list.txt", "wb") as fpf:   #Pickling
    pickle.dump(img_list, fpf)
with open("val_annotations.txt", "wb") as fpf1:   #Pickling
    pickle.dump(list_arr, fpf1)

# convert annotations into .tfrecord format...
!python to_tfrecord.py \
  --image_list 'val_img_list.txt' \
  --annotations 'val_annotations.txt' \
  --output_file val.tfrecord

100%|██████████| 6380/6380 [00:00<00:00, 2187318.91it/s]


I0302 21:35:31.617282 4573896128 to_tfrecord.py:101] Class mapping loaded
I0302 21:35:31.623798 4573896128 to_tfrecord.py:115] Image list loaded: 6380
100%|███████████████████████████████████████| 6380/6380 [01:04<00:00, 99.51it/s]
I0302 21:36:35.740230 4573896128 to_tfrecord.py:126] Done


In [7]:
test = imdb.get_detection_data(image_set = 'test', 
                                method = 'yolo3', 
                                occulusion_type = 'full', 
                                file_path = '/Users/rajarathnambalakrishnan/Documents/GitHub/Volvo-DataX/Berkeley/Data/',
                                **train_opts)

---------------------------------------------------------
Generating database for jaad
jaad database loaded from /Users/rajarathnambalakrishnan/Documents/GitHub/Volvo-DataX/Berkeley/Data/data_cache/jaad_database.pkl
Number of unique pedestrians 1049 
Number of samples 113981 
Data generated for YOLO3


In [8]:
# process the annotation data into a list...
with open('/Users/rajarathnambalakrishnan/Documents/GitHub/Volvo-DataX/Berkeley/Data/yolo3_test.txt', 'r') as csvfile:
    list_arr = []
    reader = csv.reader(csvfile, delimiter=' ')

    for row in reader:
        list_arr.append(row)

img_list = []
for i in tqdm.tqdm(list_arr):
    img_list.append(i[0])

# saving image list for tfrecord processing 

with open("test_img_list.txt", "wb") as fpfp:   #Pickling
    pickle.dump(img_list, fpfp)
with open("test_annotations.txt", "wb") as fpfp1:   #Pickling
    pickle.dump(list_arr, fpfp1) 

# convert annotations into .tfrecord format...
!python to_tfrecord.py \
  --image_list 'test_img_list.txt' \
  --annotations 'test_annotations.txt' \
  --output_file test.tfrecord

100%|██████████| 27036/27036 [00:00<00:00, 2837838.86it/s]


I0302 21:38:03.544939 4678155712 to_tfrecord.py:101] Class mapping loaded
I0302 21:38:03.584412 4678155712 to_tfrecord.py:115] Image list loaded: 27036
100%|█████████████████████████████████████| 27036/27036 [04:35<00:00, 98.18it/s]
I0302 21:42:38.959171 4678155712 to_tfrecord.py:126] Done
